# Welcome to the xarray interactive tutorial
## Importing some packages
Run the following block of code. We will be making use of the following packages
- numpy: library for creating and manipulating arrays
- scipy: library for manipulating arrays
- matplotlib: library for plotting arrays
- xarray: library for labelling and interacting with arrays
The following block of code will import the packages, it may take a few seconds to complete.

In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import xarray as xr

# [1] Creating the noise array
We create an array of shape [31, 21, 3]

In [ ]:
Nx = 31
Ny = 21
Nc = 3

## [1a] numpy
- We will use np.random.random to create the array and fill it with random values between 0 and 1
- The display function is used to see a representation of the array

In [ ]:
noise_shape = [Nx, Ny, Nc]
noise_src = np.random.random(noise_shape)
display(noise_src)

## [1b] Physical dimensions of the image
We will define coordinates to map onto the array
- Width: 3 m
- Height: 2 m
- Color: RGB
The coordinate vectors of width and height will be generated using the np.linspace function

In [ ]:
width = 3
height = 2
wcoord = np.linspace(0, width, Nx)
hcoord = np.linspace(0, height, Ny)
ccoord = ['r','g','b']
display(wcoord)

## [1d] Creating an xarray DataArray
We will create a DataArray out of the numpy array and display it

In [ ]:
# Remove
noise = xr.DataArray(noise_src, dims=["w","h","c"],coords = [wcoord, hcoord,ccoord])
display(noise)

# [2] Displaying the image
We will be visualizing the noise array we created. All visualization is done through the matplotlib library. 
## [2a] Plotting numpy array
Noticing a few things
- Axis Labels
- Axis Ticks

In [ ]:
plt.pcolor(noise_src)
plt.show()

## [2b] Fixing numpy plot
We can add a few things to fix the plot
- Transpose image,
- Add coordinates to plot
- Add axis labels

In [ ]:
# REMOVE
plt.pcolor(wcoord, hcoord, np.transpose(noise_src, axes=[1,0,2]))
plt.ylabel("h")
plt.xlabel("w")
plt.show()

## [2c] with xarray
As xarray bundles the coordinate vectors in a DataArray, plotting takes fewer lines of code to accomplish the same features as [2b]

In [ ]:
# REMOVE 
noise.plot.imshow(x="w")
plt.show()

# [3] Plotting RGB components
As images have a color dimension, we can view the individual RGB components of an image
## [3a] With numpy + matplotlib
Similar to the previous section
- have to transpose the matrix
- pass in the coordinate vectors.

The individual colour components are obtained with array indexing

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(10,3))
colors = ["Reds","Blues","Greens"]
for i in range(3):
    axs[i].pcolor(wcoord, hcoord, noise_src[:,:,i].T,cmap=colors[i])
    axs[i].set_ylabel("h")
    axs[i].set_xlabel("w")
plt.tight_layout()
plt.show(fig)

## [3b] xarray
We will showcase using 3 methods to isolate colour components
1. Array Indexing
2. .isel method to select based on index
3. .sel method to select based on value

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(10,3))
colors = ["Reds","Greens","Blues"]
# CODE HERE [REMOVE]
noise[:,:,0].plot.imshow(x="w",ax=axs[0],cmap=colors[0])
noise.isel(c=1).plot.imshow(x="w",ax=axs[1],cmap=colors[1])
noise.sel(c="b").plot.imshow(x="w",ax=axs[2],cmap=colors[2])
plt.tight_layout()
plt.show(fig)

# [4] Smoothing out noise
We will map our low resolution noise array onto a higher resolution grid
- New Nx: 301
- New Ny: 201

Start by defining new coordinate vectors

In [ ]:
Nx_new = 301
Ny_new = 201
new_wcoord = np.linspace(0,width,Nx_new)
new_hcoord = np.linspace(0,height,Ny_new)

## [4a] Interpolating onto higher resolution grid
We can use the interp method to interpolate our noise onto a larger grid
- method options: "linear" [Default], "cubic"

In [ ]:
# REMOVE
smooth = noise.interp(w=new_wcoord,h=new_hcoord,method="cubic")
display(smooth)

## [4c] Visualize before and after
We will plot the noise and smooth arrays side by side
- Similar to before we will use the .plot.imshow method
- We can pass in an axis variable for subplots

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(10,5))
# Code Here
noise.plot.imshow(ax=axs[0],x="w")
axs[0].set_title("Before")
# Code Here
smooth.plot.imshow(ax=axs[1],x="w")
axs[1].set_title("After")
plt.show()

## [4d] What did interpolation do?
Using the .sel method we will examine the effects of interpolation by plotting the red component at a specific height (h=1)
- Scatter plot for noise
- Lineplot for smooth

In [ ]:
# REMOVE
noise.sel(h=1.0001,method="nearest").sel(c='r').plot.scatter(color="red",label="Before")
smooth.sel(h=1.0001,method="nearest").sel(c='r').plot(color="red",label="After")
plt.legend()
plt.show()

## [4e] Access a coordinate not on the grid
h = 1.0001 is not on either the noise or smooth arrays. But we can use direct xarray to find the nearest value

In [ ]:
# REMOVE
noise.sel(h=1.0001,method="nearest").sel(c='r').plot.scatter(color="red",label="Before")
smooth.sel(h=1.0001,method="nearest").sel(c='r').plot(color="red",label="After")
plt.legend()
plt.show()

# [5] Transforming the smooth array
We will be making modifications to the smooth array. Start by creating a copy of the smooth array

In [ ]:
# Remove
tform = smooth.copy()

## [5a] Transforming the whole array
Similar to numpy, we can apply numpy functions on the grid
- Output array should still have values between 0 and 1
- Try a combination of trig/exponential functions i.e. (np.sin(np.exp(x)))
- Use the modulus operator (% 1) to limit values between 0 and 1

Plot transformed results

In [ ]:
tform = np.sin(np.exp(tform**2)) % 1

tform.plot.imshow(x="h")

## [5b] Transforming part of the array
We can make modifications to the array using the .loc attribute
In this example I will remove the green component from the image [Set to 0]

In [ ]:
tform.loc[{"c":"g"}] = 0
tform.plot.imshow(x="h")

## [5c] Accessing a range of coordinates
We can access a range of coordinates using slice objects
In this example I remove the red component from a portion of the image

In [ ]:
tform.loc[:,slice(0.25,0.75),"r"] = 0
tform.loc[{"w":slice(0.25,0.75), "c":"r"}] = 0
tform.plot.imshow(x="h")